# 05 — Export JSON for Frontend
**SweetReturns Golden City** data pipeline — **FINAL NOTEBOOK**

This notebook reads all computed data from the pipeline and exports the JSON payload that the React frontend consumes.

### Pipeline recap
| # | Notebook | Output |
|---|---|---|
| 01 | EDA | `stock_data_clean.parquet` |
| 02 | Feature Engineering | `stock_features.parquet`, `correlation_matrix.parquet` |
| 03 | Golden Tickets | `stock_tickets.parquet`, `graph_features.parquet` |
| 04 | Forward Returns | `stock_directions.parquet` |
| **05** | **Export JSON** | **`frontend_payload.json`** |

### Output schema
```json
{
  "generated_at": "2023-11-15",
  "regime": "bull",
  "stock_count": 500,
  "stocks": [ ... ],
  "correlation_edges": [ ... ],
  "sectors": [ ... ]
}
```

In [ ]:
import pandas as pd
import numpy as np
import json

df = pd.read_parquet("stock_directions.parquet")
corr_matrix = pd.read_parquet("correlation_matrix.parquet")
graph_features = pd.read_parquet("graph_features.parquet")
print(f"Loaded: {df.shape}")
print(f"Correlation matrix: {corr_matrix.shape}")
print(f"Graph features: {graph_features.shape}")

In [ ]:
# ---- Latest-date snapshot ----
# Take the most recent trading day per ticker.
# If some tickers are missing on the very latest date,
# fall back to their most recent available row.

latest_date = df["Date"].max()
snapshot = df[df["Date"] == latest_date].copy()

# If some tickers are missing on the latest date, use their most recent available
missing = set(df["ticker"].unique()) - set(snapshot["ticker"].unique())
for ticker in missing:
    ticker_data = df[df["ticker"] == ticker].sort_values("Date").iloc[-1:]
    snapshot = pd.concat([snapshot, ticker_data])

snapshot = snapshot.drop_duplicates(subset="ticker", keep="last")
print(f"Snapshot: {len(snapshot)} stocks as of {latest_date.date()}")
print(f"Back-filled from earlier dates: {len(missing)} tickers")

In [ ]:
# ---- Assign city grid positions by sector ----
# Each sector occupies a SECTOR_SIZE x SECTOR_SIZE block on the XZ plane.
# Within a sector, stocks are laid out in a grid sorted by golden_score.

SECTORS = [
    {"name": "Technology", "position": [0, 0], "color": "#00BFFF"},
    {"name": "Healthcare", "position": [1, 0], "color": "#FF69B4"},
    {"name": "Financials", "position": [2, 0], "color": "#FFD700"},
    {"name": "Consumer Discretionary", "position": [0, 1], "color": "#FF4500"},
    {"name": "Communication Services", "position": [1, 1], "color": "#9370DB"},
    {"name": "Industrials", "position": [2, 1], "color": "#7FFF00"},
    {"name": "Consumer Staples", "position": [0, 2], "color": "#FFA500"},
    {"name": "Energy", "position": [1, 2], "color": "#DC143C"},
    {"name": "Utilities", "position": [2, 2], "color": "#40E0D0"},
    {"name": "Real Estate", "position": [0, 3], "color": "#DDA0DD"},
    {"name": "Materials", "position": [1, 3], "color": "#8B4513"},
]

SECTOR_SIZE = 50
sector_map = {s["name"]: s for s in SECTORS}

def assign_city_positions(snapshot):
    positions = {}
    for sector_info in SECTORS:
        name = sector_info["name"]
        sx, sy = sector_info["position"]
        base_x = sx * SECTOR_SIZE - 75
        base_z = sy * SECTOR_SIZE - 75

        sector_stocks = snapshot[snapshot["sector"] == name].sort_values(
            "golden_score", ascending=False
        )
        n = len(sector_stocks)
        cols = int(np.ceil(np.sqrt(n)))

        for i, (idx, row) in enumerate(sector_stocks.iterrows()):
            col = i % cols
            row_num = i // cols
            spacing = SECTOR_SIZE / (cols + 1)
            x = base_x + (col + 1) * spacing
            z = base_z + (row_num + 1) * spacing
            positions[row["ticker"]] = {
                "x": round(float(x), 2),
                "y": 0,
                "z": round(float(z), 2),
            }
    return positions

city_positions = assign_city_positions(snapshot)
print(f"Assigned positions to {len(city_positions)} stocks")

# Show a few examples
for t in list(city_positions.keys())[:5]:
    print(f"  {t}: {city_positions[t]}")

In [ ]:
# ---- Build stocks JSON array ----
# Each stock entry follows the API schema consumed by the React frontend.

stocks_json = []
for _, row in snapshot.iterrows():
    ticker = row["ticker"]
    pos = city_positions.get(ticker, {"x": 0, "y": 0, "z": 0})

    stock = {
        "ticker": ticker,
        "company": row.get("company", ticker),
        "sector": row.get("sector", "Unknown"),
        "golden_score": int(row.get("golden_score", 0)),
        "is_platinum": bool(row.get("is_platinum", False)),
        "ticket_levels": {
            "dip_ticket": bool(row.get("ticket_1_dip", 0)),
            "shock_ticket": bool(row.get("ticket_2_shock", 0)),
            "asymmetry_ticket": bool(row.get("ticket_3_asymmetry", 0)),
            "dislocation_ticket": bool(row.get("ticket_4_dislocation", 0)),
            "convexity_ticket": bool(row.get("ticket_5_convexity", 0)),
        },
        "direction_bias": {
            "buy": round(float(row.get("buy_pct", 0.25)), 3),
            "call": round(float(row.get("call_pct", 0.25)), 3),
            "put": round(float(row.get("put_pct", 0.25)), 3),
            "short": round(float(row.get("short_pct", 0.25)), 3),
        },
        "forward_return_distribution": {
            "p5": round(float(row.get("fwd_p5", 0)), 4),
            "p25": round(float(row.get("fwd_p25", 0)), 4),
            "median": round(float(row.get("fwd_median", 0)), 4),
            "p75": round(float(row.get("fwd_p75", 0)), 4),
            "p95": round(float(row.get("fwd_p95", 0)), 4),
        },
        "city_position": pos,
        "store_dimensions": {
            "width": round(float(row.get("store_width", 3)), 1),
            "height": round(float(row.get("store_height", 2)), 1),
            "depth": round(float(row.get("store_depth", 3)), 1),
        },
        "brand_color": sector_map.get(row.get("sector"), {}).get("color", "#FFFFFF"),
        "drawdown_current": round(float(row.get("drawdown_pct", 0)), 4),
        "volume_percentile": round(float(row.get("volume_percentile", 0.5)), 3),
        "volatility_percentile": round(float(row.get("vol_percentile", 0.5)), 3),
        "rarity_percentile": round(float(row.get("rarity_percentile", 0.5)), 3),
    }
    stocks_json.append(stock)

print(f"Built {len(stocks_json)} stock entries")
print(f"\nSample keys: {list(stocks_json[0].keys())}")

In [ ]:
# ---- Build correlation edges JSON ----
# Only include edges where |weight| > threshold to keep
# the payload size manageable for the frontend.

edges_json = []
threshold = 0.5
tickers = corr_matrix.columns.tolist()
for i, t1 in enumerate(tickers):
    for j, t2 in enumerate(tickers):
        if j > i:
            w = float(corr_matrix.iloc[i, j])
            if abs(w) > threshold:
                edges_json.append({
                    "source": t1,
                    "target": t2,
                    "weight": round(w, 4),
                })

print(f"Correlation edges (|w| > {threshold}): {len(edges_json)}")
print(f"Total possible pairs: {len(tickers) * (len(tickers) - 1) // 2}")
print(f"Density: {len(edges_json) / (len(tickers) * (len(tickers) - 1) // 2) * 100:.1f}%")

In [ ]:
# ---- Build final payload and export ----

payload = {
    "generated_at": str(latest_date),
    "regime": snapshot["regime_label"].mode().iloc[0] if "regime_label" in snapshot.columns else "unknown",
    "stock_count": len(stocks_json),
    "stocks": stocks_json,
    "correlation_edges": edges_json,
    "sectors": SECTORS,
}

# Export JSON (pretty-printed)
with open("frontend_payload.json", "w") as f:
    json.dump(payload, f, indent=2, default=str)

# Also save a compact version
with open("frontend_payload.min.json", "w") as f:
    json.dump(payload, f, separators=(",", ":"), default=str)

import os
size_kb = os.path.getsize("frontend_payload.json") / 1024
print(f"Exported frontend_payload.json: {size_kb:.1f} KB")
print(f"Exported frontend_payload.min.json: {os.path.getsize('frontend_payload.min.json') / 1024:.1f} KB")
print(f"\nSample stock entry:")
print(json.dumps(stocks_json[0], indent=2))

In [ ]:
# ---- Optional: copy to project public folder ----
# Uncomment to copy to your project's public folder:
# import shutil
# shutil.copy("frontend_payload.json", "/path/to/wow-street/public/data/frontend_payload.json")
print("Done! Copy frontend_payload.json to your project's public/data/ folder")
print("The React frontend will load this at /data/frontend_payload.json")

## Pipeline Summary

The SweetReturns Golden City data pipeline transforms raw OHLCV data into a structured JSON payload for an interactive 3D city visualization.

### Notebook-by-notebook

| # | Notebook | What it does | Key output |
|---|---|---|---|
| 01 | **EDA** | Downloads data, profiles 500 tickers, maps GICS sectors, plots distributions & correlations | `stock_data_clean.parquet` |
| 02 | **Feature Engineering** | Computes rolling volatility, drawdowns, volume percentiles, forward returns, regime labels | `stock_features.parquet`, `correlation_matrix.parquet` |
| 03 | **Golden Tickets** | Scores 5 ticket dimensions (dip, shock, asymmetry, dislocation, convexity), computes `golden_score`, flags platinum stocks, builds graph features | `stock_tickets.parquet`, `graph_features.parquet` |
| 04 | **Forward Returns** | Assigns direction bias (buy/call/put/short), computes store dimensions & agent density | `stock_directions.parquet` |
| 05 | **Export JSON** | Snapshots latest date, lays out the city grid, assembles full API payload | `frontend_payload.json` |

### Frontend integration

The React app expects `frontend_payload.json` at `/data/frontend_payload.json`. The payload includes:

- **stocks[]** — each stock's score, tickets, direction bias, store geometry, brand color, and city position
- **correlation_edges[]** — significant pairwise correlations rendered as connecting roads/pipes
- **sectors[]** — sector metadata (name, grid position, color) for district labeling
- **regime** — current market regime label for global theming

### Reprocessing

To refresh the data, re-run notebooks 01–05 in order. Only notebook 05 needs to be re-run if the pipeline logic hasn't changed and you just want a fresh date snapshot.